#### 2-1. Nate 뉴스기사 제목 스크래핑하기
https://news.nate.com/recent?mid=n0100   
최신뉴스, 정치 , 경제, 사회, 세계, IT/과학 6개의 섹션의 뉴스를 출력하는 함수를 생성하여 스크래핑 하기


In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin  # url 합치는 도구
from IPython.display import Image, display # 이미지를 한 번에 보여주는 도구

In [3]:
section_dict = {'최신뉴스': 'n0100','정치': 'n0200',
                '경제': 'n0300','사회': 'n0400',
                '세계': 'n0500','IT/과학': 'n0600'}

In [4]:
def scrap_nate_news(section_name):
    mid = section_dict.get(section_name)
    url = f"https://news.nate.com/recent?mid={mid}"
    
    req_header = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }

    res = requests.get(url, headers=req_header)
    res.encoding = 'euc-kr' # 네이트는 이거?

    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
        print(f"======> {url} [{section_name}] 뉴스 <======")

        # <div class="mduSubjectList"> 가 하나의 기사 덩어리
        articles = soup.select("div.mduSubjectList")

        for article in articles:
            # 이미지 처리
            img_tag = article.select_one("span.ib img")
            
            if img_tag:
                img_src = img_tag['src']
                # //thumbnews... 로 시작하므로 urljoin 필수
                full_img_url = urljoin(url, img_src)
                
                print("[이미지]")
                display(Image(url=full_img_url, width=120))
            else:
                pass # 이미지 없으면 패스

            # 제목 은 h2의 tit
            title_tag = article.select_one("h2.tit")
            
            # 링크는 a 태그의 lt1
            link_tag = article.select_one("a.lt1")

            if title_tag and link_tag:
                title = title_tag.text.strip()
                link = link_tag['href']  # href 속성에 링크만을 가져오자
                full_link = urljoin(url, link)

                print(f"[제목] {title}")
                print(f"[링크] {full_link}")
                print("----------------------------------------------------")

    else:
        print(f"Error Code: {res.status_code}")

In [5]:
scrap_nate_news('사회')

======> https://news.nate.com/recent?mid=n0400 [사회] 뉴스 <======
[이미지]


[제목] 가평서 코브라 헬기 추락해 2명 사망
[링크] https://news.nate.com/view/20260209n15873?mid=n0400
----------------------------------------------------
[이미지]


[제목] "설 연휴 주차? 학교 운동장 이용하세요"
[링크] https://news.nate.com/view/20260209n18970?mid=n0400
----------------------------------------------------
[이미지]


[제목] 가평서 훈련 중 군 헬기 추락
[링크] https://news.nate.com/view/20260209n18844?mid=n0400
----------------------------------------------------
[이미지]


[제목] 서산문화재단-단국대학교 정책경영대학원, 인재 양성 위한 업무협약 체결
[링크] https://news.nate.com/view/20260209n18966?mid=n0400
----------------------------------------------------
[이미지]


...

[제목] [속보] 육군 "비상 절차 훈련 중 추락…헬기 탑승자 2명 숨져"
[링크] https://news.nate.com/view/20260209n18634?mid=n0400
----------------------------------------------------
[이미지]


[제목] [르포] "포탄 터진 줄 알았다" 가평 민가 30m 옆 헬기 추락…준위 2명 사망 '충격'
[링크] https://news.nate.com/view/20260209n18944?mid=n0400
----------------------------------------------------
[이미지]


[제목] 설 떡국용 손만두 포장
[링크] https://news.nate.com/view/20260209n18941?mid=n0400
----------------------------------------------------
[이미지]


[제목] 경기도, 가축전염병 예방 및 축산물 유통에 956억 원 투입
[링크] https://news.nate.com/view/20260209n18936?mid=n0400
----------------------------------------------------
[이미지]


[제목] '강제추행 혐의' 김보협 전 조국당 대변인…혐의 부인
[링크] https://news.nate.com/view/20260209n18935?mid=n0400
----------------------------------------------------


#### 2-2. 하나의 네이버 웹툰과 1개의 회차에 대한 Image 다운로드 하기


In [6]:
import os

In [7]:

def download_one_episode(title,no,url):

    req_header = {
        'referer': url,
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
    }
    res = requests.get(url, headers=req_header)

    if res.ok:
        # 이미지 url 추출
        soup = BeautifulSoup(res.text, 'html.parser')
        img_tags = soup.select("img[src*='IMAG01']") # img 태그의 src 속성에서 'IMG01' 포함한 모든 태그 추출


        #리스트 컴프리헨션
        img_url_list = [img_tag['src'] for img_tag in img_tags]  
        # img_url_list2 에 모든 이미지 링크가 배열됨

        dir_path = os.path.join('img', title, str(no))
        if not os.path.exists(dir_path): 
            os.makedirs(dir_path)

        for img_url in img_url_list:
            res = requests.get(img_url, headers=req_header)  #header를 설정 안하면 403(권한 없음이 뜬다)
            img_data = res.content

            # url에서 파일명만 추출하기
            # img/xxxIMG01.jpg
            file_path = os.path.join(dir_path, os.path.basename(img_url))

            # binary 데이터를 file에 write 하기
            with open(file_path,'wb') as file:
                print(f'Writing to {file_path}({len(img_data):,}bytes)')
                file.write(img_data)

    else:
        print(f'Error Code = {res.status_code}')




In [8]:
# 실행 테스트
download_one_episode('엄마를 만나러 가는 길', 1, 'https://comic.naver.com/webtoon/detail?titleId=834369&no=1&week=finish')

Writing to img\엄마를 만나러 가는 길\1\20241125101030_b19e80838a6e85b8aca65b3b02d47b28_IMAG01_1.jpg(31,242bytes)
Writing to img\엄마를 만나러 가는 길\1\20241125101030_b19e80838a6e85b8aca65b3b02d47b28_IMAG01_2.jpg(73,417bytes)
Writing to img\엄마를 만나러 가는 길\1\20241125101030_b19e80838a6e85b8aca65b3b02d47b28_IMAG01_3.jpg(101,481bytes)
Writing to img\엄마를 만나러 가는 길\1\20241125101030_b19e80838a6e85b8aca65b3b02d47b28_IMAG01_4.jpg(81,404bytes)
Writing to img\엄마를 만나러 가는 길\1\20241125101030_b19e80838a6e85b8aca65b3b02d47b28_IMAG01_5.jpg(172,033bytes)
Writing to img\엄마를 만나러 가는 길\1\20241125101030_b19e80838a6e85b8aca65b3b02d47b28_IMAG01_6.jpg(111,113bytes)
Writing to img\엄마를 만나러 가는 길\1\20241125101030_b19e80838a6e85b8aca65b3b02d47b28_IMAG01_7.jpg(50,877bytes)
Writing to img\엄마를 만나러 가는 길\1\20241125101030_b19e80838a6e85b8aca65b3b02d47b28_IMAG01_8.jpg(172,144bytes)
Writing to img\엄마를 만나러 가는 길\1\20241125101030_b19e80838a6e85b8aca65b3b02d47b28_IMAG01_9.jpg(96,121bytes)
Writing to img\엄마를 만나러 가는 길\1\20241125101030_b19e80838a6e85b